In [1]:
import numpy as np 
import pandas as pd

In [64]:
folder_path = 'SQP - Monthly/'

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

files = []
for file in csv_files:
    df_temp = pd.read_csv(file, header=1)
    files.append(df_temp)

combined_df = pd.concat(files, ignore_index=True)

In [67]:
combined_df.shape

(800, 34)

In [84]:
import pandas as pd
from datetime import date

def fill_missing_dates(data: pd.DataFrame, date_column: str,  freq: str) -> pd.DataFrame:
    """
    Fills missing dates in a time series DataFrame for each unique product,
    with a frequency of 'M' (monthly) or 'W' (weekly).

    Args:
        data: A pandas DataFrame with at least 'Date' and 'Product' columns.
              The 'Date' column should be of type datetime64[ns].
        start_date: The desired start date of the time series.
        end_date: The desired end date of the time series.
        freq: The frequency of the time series. Must be either 'M' or 'W'.

    Returns:
        A new DataFrame with all missing dates filled for each product,
        with missing data for other columns set to NaN.
        
    Raises:
        ValueError: If the provided frequency is not 'M' or 'W'.
    """
    # Validate the frequency input
    if freq not in ['ME', 'W']:
        raise ValueError("Invalid frequency. Please use 'M' for monthly or 'W' for weekly.")

    data = data.rename(columns = {'Reporting Date': 'Date'})
    # Ensure the 'Date' column is in the correct format
    data['Date'] = pd.to_datetime(data['Date'])

    start_date = data['Date'].min()
    end_date = data['Date'].max()

    # Get a list of all unique products
    products = data['Search Query'].unique()

    # Generate a complete date range based on the specified frequency
    full_date_range = pd.date_range(start=start_date, end=end_date, freq=freq)

    # Create an empty list to store the processed data for each product
    filled_data_list = []

    # Iterate over each product
    for product in products:
        # Filter the original data for the current product
        product_data = data[data['Search Query'] == product].copy()

        # Create a DataFrame with the full date range for this product
        full_product_df = pd.DataFrame({
            'Date': full_date_range,
            'Search Query': product
        })

        # Merge the full date range with the product's data. This will
        # automatically fill in missing dates with NaNs for other columns.
        merged_df = pd.merge(full_product_df, product_data, on=['Date', 'Search Query'], how='left')

        # Append the merged DataFrame to our list
        filled_data_list.append(merged_df)

    # Concatenate all the individual product DataFrames into one
    filled_df = pd.concat(filled_data_list, ignore_index=True)

    # Sort the final DataFrame by Product and then by Date for clean display
    filled_df.sort_values(by=['Search Query', 'Date'], inplace=True)

    return filled_df

full_df = fill_missing_dates(combined_df, 'Reporting Date', 'ME').reset_index(drop = True).fillna(0)

search_query = 'three farmer'
filtered_df = full_df[full_df['Search Query']==search_query].reset_index(drop = True)
filtered_df = filtered_df.sort_values(by = 'Date', ascending = True)
required_columns = ['Date','Search Query Score', 'Search Query Volume', 'Purchases: Total Count', 'Impressions: ASIN Share %', 'Clicks: Click Rate %','Clicks: ASIN Share %','Cart Adds: Cart Add Rate %','Cart Adds: ASIN Share %','Purchases: Purchase Rate %','Purchases: ASIN Share %']
filtered_df = filtered_df[required_columns]
filtered_df.columns = ['Date','Search Query Score', 'Search Query Volume', 'Total Purchase Count', 'Brand Impressions Share', 'Click Rate','Brand Click Share','Cart Add Rate','Brand Cart Adds Share','Purchase Rate','Brand Purchase Share']

filtered_df

,Date,Search Query Score,Search Query Volume,Total Purchase Count,Brand Impressions Share,Click Rate,Brand Click Share,Cart Add Rate,Brand Cart Adds Share,Purchase Rate,Brand Purchase Share
0,2025-01-31,82.0,25.0,0.0,7.16,60.00,46.67,16.00,75.00,0.00,0.0
1,2025-02-28,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0
2,2025-03-31,64.0,32.0,1.0,7.03,59.38,26.32,15.63,60.00,3.13,100.0
3,2025-04-30,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0
4,2025-05-31,36.0,41.0,5.0,6.61,65.85,33.33,31.71,30.77,12.20,80.0
5,2025-06-30,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0
6,2025-07-31,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0
7,2025-08-31,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0


In [98]:
import pandas as pd
import plotly.graph_objects as go

def create_interactive_plots(filtered_df):
    """
    Creates four interactive plots from a pandas DataFrame, with each plot
    having its own legend.

    Args:
        filtered_df (pd.DataFrame): The input DataFrame with the required columns.
    """
    # Ensure the 'Date' column is in datetime format for proper plotting
    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])

    # --- Plot 1: Brand Purchase Share, Brand Click Share, and Search Query Volume ---
    fig1 = go.Figure()
    fig1.add_trace(
        go.Bar(
            x=filtered_df['Date'],
            y=filtered_df['Search Query Volume'],
            name='Search Query Volume',
            marker_color='turquoise'
        )
    )
    fig1.add_trace(
        go.Scatter(
            x=filtered_df['Date'],
            y=filtered_df['Brand Purchase Share'],
            mode='lines+markers',
            name='Brand Purchase Share',
            line=dict(color='darkgreen'),
            yaxis='y2'
        )
    )
    fig1.add_trace(
        go.Scatter(
            x=filtered_df['Date'],
            y=filtered_df['Brand Click Share'],
            mode='lines+markers',
            name='Brand Click Share',
            line=dict(color='saddlebrown'),
            yaxis='y2'
        )
    )
    fig1.update_layout(
        title_text='Brand Purchase Share, Brand Click Share, and Search Query Volume',
        title_x=0.5,
        xaxis_title='Date',
        yaxis_title='Search Query Volume',
        yaxis2=dict(
            title='Share %',
            overlaying='y',
            side='right'
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig1.show()


    # --- Plot 2: Brand Cart Adds Share and Brand Click Share ---
    fig2 = go.Figure()
    fig2.add_trace(
        go.Scatter(
            x=filtered_df['Date'],
            y=filtered_df['Brand Cart Adds Share'],
            mode='lines+markers',
            name='Brand Cart Adds Share',
            line=dict(color='saddlebrown')
        )
    )
    fig2.add_trace(
        go.Scatter(
            x=filtered_df['Date'],
            y=filtered_df['Brand Click Share'],
            mode='lines+markers',
            name='Brand Click Share',
            line=dict(color='royalblue')
        )
    )
    fig2.update_layout(
        title_text='Brand Cart Adds Share and Brand Click Share',
        title_x=0.5,
        xaxis_title='Date',
        yaxis_title='Share %',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig2.show()


    # --- Plot 3: Brand Impressions Share and Search Query Volume ---
    fig3 = go.Figure()
    fig3.add_trace(
        go.Bar(
            x=filtered_df['Date'],
            y=filtered_df['Search Query Volume'],
            name='Search Query Volume',
            marker_color='turquoise'
        )
    )
    fig3.add_trace(
        go.Scatter(
            x=filtered_df['Date'],
            y=filtered_df['Brand Impressions Share'],
            mode='lines+markers',
            name='Brand Impressions Share',
            line=dict(color='saddlebrown'),
            yaxis='y2'
        )
    )
    fig3.update_layout(
        title_text='Brand Impressions Share and Search Query Volume',
        title_x=0.5,
        xaxis_title='Date',
        yaxis_title='Search Query Volume',
        yaxis2=dict(
            title='Share %',
            overlaying='y',
            side='right'
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig3.show()


    # --- Plot 4: Brand Purchase Share and Brand Cart Adds Share ---
    fig4 = go.Figure()
    fig4.add_trace(
        go.Scatter(
            x=filtered_df['Date'],
            y=filtered_df['Brand Purchase Share'],
            mode='lines+markers',
            name='Brand Purchase Share',
            line=dict(color='saddlebrown')
        )
    )
    fig4.add_trace(
        go.Scatter(
            x=filtered_df['Date'],
            y=filtered_df['Brand Cart Adds Share'],
            mode='lines+markers',
            name='Brand Cart Adds Share',
            line=dict(color='royalblue')
        )
    )
    fig4.update_layout(
        title_text='Brand Purchase Share and Brand Cart Adds Share',
        title_x=0.5,
        xaxis_title='Date',
        yaxis_title='Share %',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig4.show()


create_interactive_plots(filtered_df)


In [1]:
## Video 2 analysis

In [46]:
all_sheets = pd.read_excel(
    'Ad reports/Sponsored product/3F_SP_Ad_product_report_-_Aug_1_-_31,_25.xlsx',
    sheet_name=None  # returns dictionary of all sheets
)
print("Available sheets:", list(all_sheets.keys()))

df_ad_product = pd.read_excel('Ad reports/Sponsored product/3F_SP_Ad_product_report_-_Aug_1_-_31,_25.xlsx', sheet_name='3F_SP_Ad_product_report_-_Aug_1')
df_ad_product.head(3)

Available sheets: ['Sheet1', '3F_SP_Ad_product_report_-_Aug_1']


,Start Date,End Date,Portfolio name,Currency,Campaign Name,Ad Group Name,Country,Advertised SKU,Advertised ASIN,Impressions,...,7 Day Total Sales ($),Total Advertising Cost of Sales (ACoS),Total Return on Advertising Spend (ROAS),7 Day Total Orders (#),7 Day Total Units (#),7 Day Conversion Rate,7 Day Advertised SKU Units (#),7 Day Other SKU Units (#),7 Day Advertised SKU Sales ($),7 Day Other SKU Sales ($)
0,2025-08-01,2025-08-31,Camelina,CAD,NB | SP | Camelina | KW,Camelina KW,Canada,C6009,B07LC2XLYL,67470,...,1892.50,0.279826,3.573654,51,51,0.174061,47,4,1639.54,252.96
1,2025-08-06,2025-08-06,Brand,CAD,B || SP | KW,Chickpeas,Canada,C70025,B07MFZK2MY,0,...,0.00,NaN,NaN,0,0,NaN,0,0,0.00,0.00
2,2025-08-01,2025-08-31,Brand,CAD,B || SP | KW,Camelina,Canada,C6009,B07LC2XLYL,9004,...,164.95,0.370658,2.697906,5,5,0.135135,4,1,139.96,24.99


In [ ]:
unique_asin = df_ad_product['Advertised ASIN'].unique()
selected_asin = 'B0BH6G8Q94'
campaigns = df_ad_product[df_ad_product['Advertised ASIN'] == selected_asin]['Campaign Name'].unique()
st_imp_df = pd.read_csv('Ad reports/Sponsored product/3F_Canada_SP_ST_imp_share_report_-_Aug_1_-_31,_25.csv')
filtered_st_imp_df = st_imp_df[st_imp_df['Campaign Name'].isin(campaigns)]

filtered_st_imp_df_numeric = filtered_st_imp_df.copy()

# Also ensure other columns are numeric
numeric_columns = ['Search Term Impression Rank','Search Term Impression Share','Impressions', 'Clicks', 'Spend', '7 Day Total Orders (#)', '7 Day Total Sales ($)']
for col in numeric_columns:
    filtered_st_imp_df_numeric[col] = filtered_st_imp_df_numeric[col].astype(str).str.replace('$', '', regex=False)
    filtered_st_imp_df_numeric[col] = filtered_st_imp_df_numeric[col].astype(str).str.replace('%', '', regex=False)
    filtered_st_imp_df_numeric[col] = pd.to_numeric(filtered_st_imp_df_numeric[col], errors='coerce')

grouped_filtered_st_imp_df = filtered_st_imp_df_numeric.groupby(['Customer Search Term']).agg({
    'Search Term Impression Rank':'mean',
    'Search Term Impression Share':'mean',
    'Impressions':'sum',
    'Clicks':'sum',
    'Spend':'sum',
    '7 Day Total Orders (#)':'sum',
    '7 Day Total Sales ($)':'sum'}).reset_index()

grouped_filtered_st_imp_df['Search Term Impression Share'] = grouped_filtered_st_imp_df['Search Term Impression Share']/100
    
grouped_filtered_st_imp_df.head(3)

,Customer Search Term,Search Term Impression Rank,Search Term Impression Share,Impressions,Clicks,Spend,7 Day Total Orders (#),7 Day Total Sales ($)
0,100 calorie pack snacks,1.0,1.0000,1,1,0.88,0,0.0
1,100 calorie snack packs,2.0,0.3077,9,2,2.16,0,0.0
2,100% gluten free snacks,1.0,0.6667,2,2,2.01,0,0.0


In [61]:
grouped_filtered_st_imp_df[grouped_filtered_st_imp_df['Customer Search Term']=='three farmers']

,Customer Search Term,Search Term Impression Rank,Search Term Impression Share,Impressions,Clicks,Spend,7 Day Total Orders (#),7 Day Total Sales ($)
583,three farmers,1.0,0.9999,78201,814,528.66,138,4231.99


In [63]:
import numpy as np

new_df = grouped_filtered_st_imp_df.copy()
new_df['Available impression'] = np.where(
    new_df['Search Term Impression Share'] != 0, 
    new_df['Impressions'] / new_df['Search Term Impression Share'], 
    None
)

new_df['CTR'] = np.where(
    new_df['Available impression'] != 0, 
    new_df['Clicks']*100 / new_df['Available impression'], 
    None
)

new_df['ACoS'] = np.where(
    new_df['7 Day Total Sales ($)'] != 0, 
    new_df['Spend']*100 / new_df['7 Day Total Sales ($)'], 
    None
)

new_df['ACR'] = np.where(
    new_df['Clicks'] != 0, 
    new_df['7 Day Total Orders (#)']*100/ new_df['Clicks'], 
    None
)

new_df['Search Term Impression Share'] = new_df['Search Term Impression Share']*100
new_df = new_df[['Customer Search Term','Search Term Impression Rank','Search Term Impression Share','Impressions','CTR', '7 Day Total Orders (#)','ACoS','ACR']]
new_df.columns = ['Search Term','Search Term Impression Rank','Search Term Impression Share','Impressions','CTR', 'Total Orders','ACoS','ACR']
new_df.head()

,Search Term,Search Term Impression Rank,Search Term Impression Share,Impressions,CTR,Total Orders,ACoS,ACR
0,100 calorie pack snacks,1.0,100.00,1,100.0,0,None,0.0
1,100 calorie snack packs,2.0,30.77,9,6.837778,0,None,0.0
2,100% gluten free snacks,1.0,66.67,2,66.67,0,None,0.0
3,3 farmers,1.0,100.00,103,27.184466,4,1.700567,14.285714
4,3 farmers chickpeas,1.0,100.00,11,36.363636,0,None,0.0
